In [2]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoConfig, BatchEncoding, BertModel,LayoutLMModel
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
model = LayoutLMModel.from_pretrained("microsoft/layoutlm-base-uncased")
config = AutoConfig.from_pretrained("microsoft/layoutlm-base-uncased")
config.hidden_size

Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing LayoutLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768

In [38]:
# import json
# result = json.load(open('./data/raw/truong_raw.jsonl'))
# result['boxes']

In [4]:
import json
from PIL import Image
result = json.load(open('./data/processed/train/huy.jsonl'))
result = json.load(open('data/processed/train/trinh.jsonl'))
result = json.load(open('data/processed/train/kha.jsonl'))
result = json.load(open('data/processed/train/linh.jsonl'))
# result = json.load(open('data/processed/trinh.jsonl'))
# result = json.load(open('data/processed/trinh.jsonl'))
words = result['text']

In [9]:
def handle_input(words, coord, size_):
        w,h = size_
        bboxes = tuple([[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in coord])
        normalized_word_boxes = bboxes
        encoding = tokenizer(' '.join(words), return_tensors="pt")
        token_boxes = []
        tokens = []
        tokens = [tokenizer.cls_token] + tokens
        i =0
        maps = []
        maps.extend([[i]])
        for word, box in zip(words, normalized_word_boxes):
            word_tokens = tokenizer.tokenize(word)
            token_boxes.extend([box] * len(word_tokens))
            tokens.extend(word_tokens)
            i += 1
            maps.extend([[i]*len(word_tokens)])

        tokens += [tokenizer.sep_token]
        maps.extend([[i+1]])

        input_ids = encoding["input_ids"]
        attention_mask = encoding["attention_mask"]
        token_type_ids = encoding["token_type_ids"]
        bbox = torch.tensor([token_boxes])
        return {'input_ids': input_ids, 'attention_mask': attention_mask,'token_type_ids':token_type_ids,'bbox':bbox , 'maps': maps}

In [12]:
w, h = Image.open(result['data_id']).size
w,h
# int(2.3)
input_ = handle_input(words,result['coord'],(w,h))
outputs = model(
    input_ids=input_['input_ids'], bbox=input_['bbox'], attention_mask=input_['attention_mask'], token_type_ids=input_['token_type_ids']
)

RuntimeError: The size of tensor a (148) must match the size of tensor b (146) at non-singleton dimension 1

In [7]:
bboxes = tuple([[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in result['coord']])
# bboxes = tuple(bboxes)
bboxes

# text label data_id coord

([436, 33, 474, 81],
 [448, 81, 481, 129],
 [474, 33, 523, 81],
 [492, 81, 525, 129],
 [525, 33, 582, 85],
 [582, 33, 661, 81],
 [293, 37, 373, 88],
 [373, 37, 431, 85],
 [661, 37, 723, 88],
 [725, 37, 784, 85],
 [525, 85, 565, 118],
 [403, 88, 448, 125],
 [370, 125, 476, 162],
 [568, 88, 622, 122],
 [624, 88, 673, 129],
 [72, 133, 89, 159],
 [476, 129, 579, 162],
 [373, 162, 502, 196],
 [516, 162, 593, 196],
 [584, 133, 615, 162],
 [615, 129, 661, 162],
 [666, 129, 713, 162],
 [612, 162, 706, 196],
 [295, 211, 394, 296],
 [690, 211, 791, 296],
 [403, 214, 537, 292],
 [551, 222, 678, 288],
 [852, 240, 946, 270],
 [462, 322, 518, 370],
 [295, 325, 373, 370],
 [375, 325, 460, 374],
 [401, 396, 744, 466],
 [295, 414, 330, 462],
 [347, 418, 394, 462],
 [368, 485, 408, 533],
 [422, 485, 467, 540],
 [467, 485, 539, 540],
 [417, 540, 542, 596],
 [485, 596, 514, 651],
 [514, 596, 570, 651],
 [298, 488, 330, 529],
 [335, 488, 366, 529],
 [549, 533, 636, 596],
 [575, 596, 748, 659],
 [291, 537, 

In [18]:
def handel(word, coord, size):
    w,h = size
    normalized_word_boxes = tuple([[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in coord])
    encoding = tokenizer(" ".join(words), return_tensors="pt")
    token_boxes = []
    tokens = []
    tokens = [tokenizer.cls_token] + tokens
    i =0
    maps = []
    maps.extend([[i]])

    for word, box in zip(words, normalized_word_boxes):
        word_tokens = tokenizer.tokenize(word)
        token_boxes.extend([box] * len(word_tokens))
        tokens.extend(word_tokens)
        i += 1
        maps.extend([[i]*len(word_tokens)])

    tokens += [tokenizer.sep_token]
    maps.extend([[i+1]])

    token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

    input_ids = encoding["input_ids"]
    attention_mask = encoding["attention_mask"]
    token_type_ids = encoding["token_type_ids"]
    bbox = torch.tensor([token_boxes])
    return {'input_ids': input_ids, 'attention_mask': attention_mask,'token_type_ids':token_type_ids,'bbox':bbox , 'maps': maps}

handel(words,result['coord'],(w,h))

{'input_ids': tensor([[  101,  1060,  2050,  5001,  7570,  2072, 10722, 14684, 12835, 12995,
          26478,  7570,  2050, 19710, 15125,  2079,  1102, 10085,  6102,  7658,
           2232,  6887, 14194,  1056,  2002,  2226, 16577, 16577,  4013, 19053,
           2389,  5709, 19710, 12436,  5112, 15125,  2064,  4907, 12731, 10085,
          26478,  1011,  4003,  6926, 12631,  2078, 25604, 28332, 24096, 21057,
          25746,  2061,  6583,  2226,  2702, 11865,  4886,  2171,  4291,  1997,
           4182,  7570, 12436, 11409,  2232,  5890,  1013,  6185,  1013,  2456,
          13012, 25311,  5003, 12835,  4710,  8254,  2232, 23755,  2080,  1018,
          22035,  2278, 21025, 10448, 14841,  2818, 10662, 11409,  2232, 19710,
           9543,  2232,  2712, 15125, 15125,  4682,  2078,  6097,  1010,  2033,
           2173,  1997,  2006,  2290, 18699, 22035,  5292, 24110,  2053,  2072,
           1011,  2053,  2072, 16215, 19098,  3070,  5292,  1060,  2050,  4682,
           2078,  1997,  60

In [8]:
normalized_word_boxes = bboxes
# print(normalized_word_boxes.shape)
# words = ["Hello", "world"]
# normalized_word_boxes = [637, 773, 693, 782], [698, 773, 733, 782]
encoding = tokenizer(" ".join(words), return_tensors="pt")
token_boxes = []
tokens = []
tokens = [tokenizer.cls_token] + tokens
i =0
maps = []
maps.extend([[i]])

for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
    tokens.extend(word_tokens)
    i += 1
    maps.extend([[i]*len(word_tokens)])

tokens += [tokenizer.sep_token]
maps.extend([[i+1]])
# add bounding boxes of cls + sep tokens

print(tokens)
token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]


# encoding = tokenizer(" ".join(words), return_tensors="pt")
input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])


print(input_ids.shape,attention_mask.shape,bbox.shape,token_type_ids.shape)
# print(pa['input_ids'].shape, pa['actual_bbox'].shape)
# sys.exit()
# token_labels = torch.tensor([1, 1, 0, 0]).unsqueeze(0)  # batch size of 1

# outputs = model(
#     input_ids=pa['input_ids'],
#     bbox=tuple(result['boxes']),
#     attention_mask=pa['attention_mask'],
#     token_type_ids=pa['token_type_ids'],
# )
outputs = model(
    input_ids=input_ids, bbox=bbox, attention_mask=attention_mask, token_type_ids=token_type_ids
)

# # last_hidden_states = outputs.last_hidden_state
# # loss = outputs.loss
# # logits = outputs.logits
last_hidden_state = outputs.last_hidden_state

# # loss = outputs.loss
# print(outputs.attentions)
print(last_hidden_state.shape)
# maps[1:-1]``

[[0], [1, 1], [2], [3, 3], [4], [5], [6, 6], [7], [8, 8], [9], [10], [11], [12, 12], [13], [14, 14], [15, 15], [16], [17, 17], [18, 18], [19, 19, 19], [20], [21], [22], [23, 23], [24], [25], [26, 26], [27], [28], [29], [30], [31, 31], [32, 32, 32, 32, 32], [33], [34, 34], [35], [36, 36], [37], [38], [39], [40], [41], [42], [43, 43], [44, 44, 44, 44, 44], [45, 45], [46], [47, 47], [48, 48], [49, 49], [50], [51, 51], [52, 52], [53, 53], [54], [55, 55], [56], [57, 57], [58], [59], [60], [61, 61], [62, 62], [63], [64], [65], [66, 66, 66], [67], [68], [69], [70, 70], [71], [72, 72], [73, 73, 73], [74], [75, 75], [76, 76], [77], [78, 78], [79], [80, 80], [81], [82, 82, 82], [83, 83, 83, 83, 83], [84, 84], [85], [86], [87], [88, 88], [89, 89, 89], [90], [91]]
['[CLS]', 'x', '##a', 'lap', 'ho', '##i', 'tu', 'chu', 'ng', '##hia', 'cong', 'ho', '##a', 'viet', 'nam', 'do', 'đ', '##oc', 'socialist', 'han', '##h', 'ph', '##uc', 't', 'he', '##u', 'nguyen', 'nguyen', 'pro', '##cic', '##al', '05', 'vi

In [137]:
import torch.nn as nn
reduce_size = 256
ln = nn.Linear(config.hidden_size, reduce_size)
outputs = ln(last_hidden_state)
last_hidden_state = outputs

In [138]:
i = 0
# reduce = torch.zeros(768)
reduce =[]
for g_token in  maps:
    ten = torch.zeros(reduce_size)
    for ele in g_token:
        # print(ele)
        ten += last_hidden_state[0][i]
        # i+=1
        # print(last_hidden_state[0][i])
        i+=1
    ten = ten/len(g_token)
    # print(ten)
    reduce.append(ten)
    # reduce = torch.cat((reduce,ten),-1)
# print(np.array(reduce).shape)
# print(reduce)
reduce=  torch.stack(reduce)
reduce.shape

torch.Size([92, 256])

In [139]:
import torch.nn as nn
class RelationTagger(nn.Module):
    def __init__(self, n_fields, hidden_size, head_p_dropout=0.1):
        super().__init__()
        self.head = nn.Linear(hidden_size, hidden_size)
        self.tail = nn.Linear(hidden_size, hidden_size)
        self.field_embeddings = nn.Parameter(
            torch.rand(1, n_fields, hidden_size))
        self.W_label_0 = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_label_1 = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, enc):

        enc_head = self.head(enc)
        enc_tail = self.tail(enc)

        batch_size = enc_tail.size(0)
        field_embeddings = self.field_embeddings.expand(batch_size, -1, -1)
        enc_head = torch.cat([field_embeddings, enc_head], dim=1)

        score_0 = torch.matmul(
            enc_head, self.W_label_0(enc_tail).transpose(1, 2))
        score_1 = torch.matmul(
            enc_head, self.W_label_1(enc_tail).transpose(1, 2))

        score = torch.cat(
            [
                score_0.unsqueeze(1),
                score_1.unsqueeze(1),
            ],
            dim=1,
        )
        return score


dropout = nn.Dropout(0.1)

In [140]:
loss = nn.CrossEntropyLoss()
rel_s = RelationTagger(
            hidden_size=reduce_size,
            n_fields=3,
        )

rel_s = rel_s(dropout(reduce.unsqueeze(0)))
# rel_s.shape
S = rel_s
# S = torch.argmax(rel_s,dim =1)
s0,s1 = S[:,:,:3,:],S[:,:,3:,:]
# torch.argmax(s0,dim =1).numpy()
s1 =  s1[:,:,1:-1,1:-1]#reduce
s0 = s0[:,:,:,1:-1]# reduce

In [141]:
rel_g = RelationTagger(
            hidden_size=reduce_size,
            n_fields=3,
        )

rel_g = rel_g(dropout(reduce.unsqueeze(0)))
# rel_s.shape
G = rel_g
# S = torch.argmax(rel_s,dim =1)
g0,g1 = G[:,:,:3,:],G[:,:,3:,:]
# torch.argmax(s0,dim =1).numpy()
g1 =  g1[:,:,1:-1,1:-1]#reduce
g0 = g0[:,:,:,1:-1]# reduce

In [142]:
graph = np.array(result['label'])
label = torch.tensor(graph[0, :3, :]).unsqueeze(0)
matrix_s = torch.tensor(graph[0, 3:, :]).unsqueeze(0)
matrix_g = torch.tensor(graph[1, 3:, :]).unsqueeze(0)
# label.shape
label.shape
loss_label_s = loss(s0, label)
loss_matrix_s = loss(s1,matrix_s)
loss_matrix_g = loss(g1,matrix_g)
print(loss_label_s, loss_matrix_s, loss_matrix_g)
loss_label_s + loss_matrix_s + loss_matrix_g



tensor(1.3106, grad_fn=<NllLoss2DBackward0>) tensor(0.8067, grad_fn=<NllLoss2DBackward0>) tensor(0.6993, grad_fn=<NllLoss2DBackward0>)


tensor(2.8166, grad_fn=<AddBackward0>)